In [ ]:
import os
os.environ['OPENAI_API_KEY']= "your_api_key_here"

In [ ]:
!pip install -q langchain_openai langchai-core requests

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

## Tools Creation

In [ ]:
from langchain_core.tools import InjectedToolArg      # we use 'InjectedToolArg' to tell our llm that do not fill this
                                  # argument instantly, I (developer/ runtime) will inject this value after running earlier tools
from typing import Annotated

In [ ]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fectches the currency conversion factor between the given base and target currencies.
  """

  url= f"https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca.pair/{base_currency}/{target_currency}"
  response= requests.get(url)
  return response.json()


@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """Given a currency conversion rate, this function calculates the target currency value from a given base currency value"""
  return base_currency_value * conversion_rate

In [ ]:
get_conversion_factor.invoke({'base_currency': 'USD', 'target_currency': 'INR'})

In [ ]:
convert.invoke({'base_currency_value': 56, 'conversion_rate': 85.125})

## Tool Binding

In [ ]:
llm= ChatOpenAi()

llm_with_tools= llm.bind_tools([get_conversion_factor, convert])

## Tool Calling

In [ ]:
query= HumanMessage("What is the conversion fcator between usd and inr, and based on that can you convert 10 usd to inr")

messages= [query]

In [ ]:
ai_message= llm_with_tools.invoke(messages)

ai_message.tool_calls

In [ ]:
messages.append(ai_message)

In [ ]:
import json

for tool_call in ai_message.tool_calls:

  if tool_call['name'] == 'get_conversion_factor':
    tool_message1= get_conversion_factor.invoke(tool_call)       # will return a ToolMessage Object
    conversionRate= json.loads(tool_message1.content)['conversion_rate']
    messages.append(tool_message1)

  elif tool_call['name'] == 'convert':
    tool_call['args']['conversion_rate']= conversionRate
    tool_message2= convert.invoke(tool_call)
    messages.append(tool_message2)

In [ ]:
messages

In [ ]:
output= llm_with_tools.invoke(messages)

print(output.content)